In [1]:
library(tidyverse)
library(repr)
library(infer)
library(cowplot)
library(broom)
library(GGally)
library(modelr)
library(car)
library(stats)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp


Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2


Attaching package: ‘modelr’


The following object is masked from ‘package:broom’:

    bootstrap


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some

In [3]:
diabetes_data <- read_csv('data/diabetes.csv')
head (diabetes_data)

Rows: 768 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, D...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
5,116,74,0,0,25.6,0.201,30,0


In [5]:
diabetes_data_clean <- diabetes_data %>%
  select(-SkinThickness) %>% # drop this column because it has too many null values
  filter(Glucose != 0 & BloodPressure != 0 & BMI != 0)
head(diabetes_data_clean)

Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6,148,72,0,33.6,0.627,50,1
1,85,66,0,26.6,0.351,31,0
8,183,64,0,23.3,0.672,32,1
1,89,66,94,28.1,0.167,21,0
0,137,40,168,43.1,2.288,33,1
5,116,74,0,25.6,0.201,30,0


In [6]:
train_indices <- sample(seq_len(nrow(diabetes_data_clean)), size = 0.8 * nrow(diabetes_data_clean))
train_data <- diabetes_data_clean[train_indices, ]
test_data <- diabetes_data_clean[-train_indices, ]

In [7]:
model <- glm(formula = Outcome ~ Age + Pregnancies + Glucose + Insulin + 
    BMI + DiabetesPedigreeFunction + Age:Pregnancies + Age:Glucose + 
    Age:Insulin, family = binomial, data = diabetes_data_clean)
model


Call:  glm(formula = Outcome ~ Age + Pregnancies + Glucose + Insulin + 
    BMI + DiabetesPedigreeFunction + Age:Pregnancies + Age:Glucose + 
    Age:Insulin, family = binomial, data = diabetes_data_clean)

Coefficients:
             (Intercept)                       Age               Pregnancies  
              -1.344e+01                 1.234e-01                 4.545e-01  
                 Glucose                   Insulin                       BMI  
               6.134e-02                -9.203e-03                 9.134e-02  
DiabetesPedigreeFunction           Age:Pregnancies               Age:Glucose  
               9.865e-01                -9.318e-03                -6.598e-04  
             Age:Insulin  
               2.360e-04  

Degrees of Freedom: 723 Total (i.e. Null);  714 Residual
Null Deviance:	    931.9 
Residual Deviance: 647.8 	AIC: 667.8

In [9]:
test_data <- test_data %>%
  mutate(predicted_prob = predict(model, newdata = ., type = "response"),
         predicted_class = ifelse(predicted_prob > 0.5, 1, 0))

test_data

Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,predicted_prob,predicted_class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,139,80,0,27.1,1.441,57,0,0.50610369,1
1,189,60,846,30.1,0.398,59,1,0.99406322,1
9,119,80,0,29.0,0.263,29,1,0.43255025,0
11,138,76,0,33.2,0.420,35,0,0.73432121,1
2,90,68,0,38.2,0.503,27,1,0.14200137,0
9,171,110,240,45.4,0.721,54,1,0.94743686,1
7,159,64,0,27.4,0.294,40,0,0.60321702,1
2,71,70,0,28.0,0.586,22,0,0.02263827,0
1,103,80,82,19.4,0.491,22,0,0.02355797,0


In [10]:
conf_matrix <- table(Predicted = test_data$predicted_class, Actual = test_data$Outcome)
accuracy <- mean(test_data$predicted_class == test_data$Outcome)

conf_matrix
accuracy

         Actual
Predicted  0  1
        0 72 25
        1 16 32

[1] 0.7172414

# Using forward selection

In [13]:
model_int <- glm(formula = Outcome ~ Age + Pregnancies + Glucose + BloodPressure + Insulin + BMI + DiabetesPedigreeFunction,
      data = diabetes_data_clean,
      family = binomial)

# use step to do forward selection
forward_model_int <- step(model_int, direction = "forward")

summary(forward_model_int)

term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-9.06,0.83,-10.95,0.00,-10.73,-7.49
Age,0.02,0.01,1.62,0.11,0.00,0.04
Pregnancies,0.12,0.03,3.46,0.00,0.05,0.18
Glucose,0.04,0.00,9.52,0.00,0.03,0.04
BloodPressure,-0.01,0.01,-1.21,0.23,-0.03,0.01
Insulin,0.00,0.00,-1.32,0.19,0.00,0.00
BMI,0.09,0.02,5.88,0.00,0.06,0.13
DiabetesPedigreeFunction,1.01,0.31,3.25,0.00,0.41,1.62


Start:  AIC=687.13
Outcome ~ Age + Pregnancies + Glucose + BloodPressure + Insulin + 
    BMI + DiabetesPedigreeFunction




Call:
glm(formula = Outcome ~ Age + Pregnancies + Glucose + BloodPressure + 
    Insulin + BMI + DiabetesPedigreeFunction, family = binomial, 
    data = diabetes_data_clean)

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -9.0568948  0.8274749 -10.945  < 2e-16 ***
Age                       0.0159550  0.0098490   1.620 0.105241    
Pregnancies               0.1161421  0.0335373   3.463 0.000534 ***
Glucose                   0.0369205  0.0038796   9.517  < 2e-16 ***
BloodPressure            -0.0105277  0.0087173  -1.208 0.227170    
Insulin                  -0.0011161  0.0008476  -1.317 0.187942    
BMI                       0.0944186  0.0160640   5.878 4.16e-09 ***
DiabetesPedigreeFunction  1.0081855  0.3098033   3.254 0.001137 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 931.94  on 723  degrees of freedom
Residual deviance

In [ ]:
forward_model <- glm (